## Functional API intro

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense,Dropout,Input,Flatten
from tensorflow.python.keras.utils.vis_utils import plot_model
from keras_visualizer import visualizer
import pandas as pd
from sklearn.model_selection import train_test_split

In [37]:
def build_model_with_functional():
    input_layer = Input(shape = (28,28))
    flatten_layer = Flatten()(input_layer)
    first_dense = Dense(128,activation='relu')(flatten_layer)
    output_layer = Dense(10,activation='softmax')(first_dense)
    
    model = tf.keras.models.Model(inputs=input_layer,outputs=output_layer)
    return model 

In [38]:
model = build_model_with_functional()
model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_9 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


## Testing this functional_api model on fashion_mnist

In [39]:
from tensorflow.keras.datasets import fashion_mnist
mnist = tf.keras.datasets.fashion_mnist
(x_train ,y_train),(x_test,y_test) = mnist.load_data()
x_train = x_train/255.0
x_test = x_test/255.0

In [41]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=5)
model.evaluate(x_test,y_test)

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1675 - accuracy: 0.9365
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1636 - accuracy: 0.9386
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1601 - accuracy: 0.9394
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1569 - accuracy: 0.9409
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.3915 - accuracy: 0.8884


[0.3915116488933563, 0.8884000182151794]

# Energy efficiency dataset : multi-output model

In [5]:
URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx'
df = pd.read_excel(URL)
print(df.head)

<bound method NDFrame.head of        X1     X2     X3      X4   X5  X6   X7  X8     Y1     Y2  Unnamed: 10  \
0    0.98  514.5  294.0  110.25  7.0   2  0.0   0  15.55  21.33          NaN   
1    0.98  514.5  294.0  110.25  7.0   3  0.0   0  15.55  21.33          NaN   
2    0.98  514.5  294.0  110.25  7.0   4  0.0   0  15.55  21.33          NaN   
3    0.98  514.5  294.0  110.25  7.0   5  0.0   0  15.55  21.33          NaN   
4    0.90  563.5  318.5  122.50  7.0   2  0.0   0  20.84  28.28          NaN   
..    ...    ...    ...     ...  ...  ..  ...  ..    ...    ...          ...   
763  0.64  784.0  343.0  220.50  3.5   5  0.4   5  17.88  21.40          NaN   
764  0.62  808.5  367.5  220.50  3.5   2  0.4   5  16.54  16.88          NaN   
765  0.62  808.5  367.5  220.50  3.5   3  0.4   5  16.44  17.11          NaN   
766  0.62  808.5  367.5  220.50  3.5   4  0.4   5  16.48  16.61          NaN   
767  0.62  808.5  367.5  220.50  3.5   5  0.4   5  16.64  16.03          NaN   

     Unna

In [6]:
df = df.dropna(axis=1)
df = df.sample(frac=1).reset_index(drop=True)

In [7]:
print(df.head())
train,test = train_test_split(df,test_size=0.2)
train_stats = train.describe()

     X1     X2     X3     X4   X5  X6    X7  X8     Y1     Y2
0  0.90  563.5  318.5  122.5  7.0   5  0.10   5  29.02  33.01
1  0.76  661.5  416.5  122.5  7.0   4  0.40   2  40.57  40.47
2  0.64  784.0  343.0  220.5  3.5   4  0.10   3  15.12  19.37
3  0.90  563.5  318.5  122.5  7.0   2  0.25   2  32.46  35.56
4  0.66  759.5  318.5  220.5  3.5   3  0.40   3  15.18  17.63


In [8]:
train_stats.pop('Y1')
train_stats.pop('Y2')
train_stats = train_stats.T

In [9]:
train_stats

,count,mean,std,min,25%,50%,75%,max
X1,614.0,0.763893,0.106457,0.62,0.66,0.74,0.82,0.98
X2,614.0,672.114007,88.721819,514.50,612.50,686.00,759.50,808.50
X3,614.0,317.382736,43.186874,245.00,294.00,318.50,343.00,416.50
X4,614.0,177.365635,45.117274,110.25,147.00,220.50,220.50,220.50
X5,614.0,5.221498,1.751195,3.50,3.50,3.50,7.00,7.00
X6,614.0,3.500000,1.118946,2.00,2.00,4.00,4.00,5.00
X7,614.0,0.237459,0.134555,0.00,0.10,0.25,0.40,0.40
X8,614.0,2.851792,1.558805,0.00,2.00,3.00,4.00,5.00


In [10]:
def format_output(data):
    Y1 = data.pop('Y1')
    Y2 = data.pop('Y2')
    Y1 = np.array(Y1)
    Y2 = np.array(Y2)
    return Y1,Y2

def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

In [11]:
train_y = format_output(train)
test_y = format_output(test)
norm_train_x = norm(train)
norm_test_x = norm(test)

In [12]:
type(train_y[1])

numpy.ndarray

In [20]:
def get_model():
    input_layer = Input(shape=(len(train .columns),))
    first_dense = Dense(units='128', activation='relu')(input_layer)
    second_dense = Dense(units='128', activation='relu')(first_dense)

    
    y1_output = Dense(units='1', name='y1_output')(second_dense)
    third_dense = Dense(units='64', activation='relu')(second_dense)

  
    y2_output = Dense(units='1', name='y2_output')(third_dense)
    
    model = tf.keras.models.Model(inputs=input_layer, outputs=[y1_output, y2_output])
    return model

In [21]:
model = get_model()
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 128)          1152        input_2[0][0]                    
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 128)          16512       dense_3[0][0]                    
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 64)           8256        dense_4[0][0]                    
_______________________________________________________________________________________

In [15]:
optimizer = tf.keras.optimizers.SGD(lr=0.001)
model.compile(optimizer=optimizer,
              loss={'y1_output': 'mse', 'y2_output': 'mse'},
              metrics={'y1_output': tf.keras.metrics.RootMeanSquaredError(),
                       'y2_output': tf.keras.metrics.RootMeanSquaredError()})

In [16]:
history = model.fit(norm_train_x, train_y,
                    epochs=500, batch_size=10, validation_data=(norm_test_x, test_y))

Epoch 1/500
62/62 [==============================] - 1s 10ms/step - loss: 223.0127 - y1_output_loss: 104.5803 - y2_output_loss: 118.4324 - y1_output_root_mean_squared_error: 10.2265 - y2_output_root_mean_squared_error: 10.8827 - val_loss: 40.8058 - val_y1_output_loss: 17.2820 - val_y2_output_loss: 23.5238 - val_y1_output_root_mean_squared_error: 4.1572 - val_y2_output_root_mean_squared_error: 4.8501
Epoch 2/500
62/62 [==============================] - 0s 3ms/step - loss: 33.3156 - y1_output_loss: 12.4229 - y2_output_loss: 20.8927 - y1_output_root_mean_squared_error: 3.5246 - y2_output_root_mean_squared_error: 4.5708 - val_loss: 30.7085 - val_y1_output_loss: 12.4871 - val_y2_output_loss: 18.2214 - val_y1_output_root_mean_squared_error: 3.5337 - val_y2_output_root_mean_squared_error: 4.2687
Epoch 3/500
62/62 [==============================] - 0s 3ms/step - loss: 24.9397 - y1_output_loss: 9.5206 - y2_output_loss: 15.4191 - y1_output_root_mean_squared_error: 3.0855 - y2_output_root_mean_sq

Epoch 22/500
62/62 [==============================] - 0s 3ms/step - loss: 13.4985 - y1_output_loss: 5.3185 - y2_output_loss: 8.1801 - y1_output_root_mean_squared_error: 2.3062 - y2_output_root_mean_squared_error: 2.8601 - val_loss: 11.2385 - val_y1_output_loss: 5.1434 - val_y2_output_loss: 6.0951 - val_y1_output_root_mean_squared_error: 2.2679 - val_y2_output_root_mean_squared_error: 2.4688
Epoch 23/500
62/62 [==============================] - 0s 3ms/step - loss: 14.1920 - y1_output_loss: 5.2120 - y2_output_loss: 8.9800 - y1_output_root_mean_squared_error: 2.2830 - y2_output_root_mean_squared_error: 2.9967 - val_loss: 34.2802 - val_y1_output_loss: 13.1348 - val_y2_output_loss: 21.1454 - val_y1_output_root_mean_squared_error: 3.6242 - val_y2_output_root_mean_squared_error: 4.5984
Epoch 24/500
62/62 [==============================] - 0s 3ms/step - loss: 12.9101 - y1_output_loss: 4.7874 - y2_output_loss: 8.1227 - y1_output_root_mean_squared_error: 2.1880 - y2_output_root_mean_squared_erro

Epoch 43/500
62/62 [==============================] - 0s 3ms/step - loss: 7.5322 - y1_output_loss: 2.3930 - y2_output_loss: 5.1392 - y1_output_root_mean_squared_error: 1.5469 - y2_output_root_mean_squared_error: 2.2670 - val_loss: 7.6916 - val_y1_output_loss: 2.4024 - val_y2_output_loss: 5.2892 - val_y1_output_root_mean_squared_error: 1.5500 - val_y2_output_root_mean_squared_error: 2.2998
Epoch 44/500
62/62 [==============================] - 0s 3ms/step - loss: 7.5835 - y1_output_loss: 2.3863 - y2_output_loss: 5.1972 - y1_output_root_mean_squared_error: 1.5448 - y2_output_root_mean_squared_error: 2.2797 - val_loss: 25.1427 - val_y1_output_loss: 4.8762 - val_y2_output_loss: 20.2664 - val_y1_output_root_mean_squared_error: 2.2082 - val_y2_output_root_mean_squared_error: 4.5018
Epoch 45/500
62/62 [==============================] - 0s 3ms/step - loss: 10.6233 - y1_output_loss: 3.4554 - y2_output_loss: 7.1679 - y1_output_root_mean_squared_error: 1.8589 - y2_output_root_mean_squared_error: 2

Epoch 64/500
62/62 [==============================] - 0s 3ms/step - loss: 4.9630 - y1_output_loss: 1.5092 - y2_output_loss: 3.4538 - y1_output_root_mean_squared_error: 1.2285 - y2_output_root_mean_squared_error: 1.8584 - val_loss: 4.2791 - val_y1_output_loss: 1.2212 - val_y2_output_loss: 3.0579 - val_y1_output_root_mean_squared_error: 1.1051 - val_y2_output_root_mean_squared_error: 1.7487
Epoch 65/500
62/62 [==============================] - 0s 3ms/step - loss: 4.2453 - y1_output_loss: 1.0811 - y2_output_loss: 3.1642 - y1_output_root_mean_squared_error: 1.0398 - y2_output_root_mean_squared_error: 1.7788 - val_loss: 3.9622 - val_y1_output_loss: 1.1025 - val_y2_output_loss: 2.8596 - val_y1_output_root_mean_squared_error: 1.0500 - val_y2_output_root_mean_squared_error: 1.6910
Epoch 66/500
62/62 [==============================] - 0s 3ms/step - loss: 4.1110 - y1_output_loss: 1.0917 - y2_output_loss: 3.0193 - y1_output_root_mean_squared_error: 1.0448 - y2_output_root_mean_squared_error: 1.73

Epoch 85/500
62/62 [==============================] - 0s 3ms/step - loss: 3.9412 - y1_output_loss: 1.0947 - y2_output_loss: 2.8465 - y1_output_root_mean_squared_error: 1.0463 - y2_output_root_mean_squared_error: 1.6872 - val_loss: 3.9083 - val_y1_output_loss: 0.7992 - val_y2_output_loss: 3.1091 - val_y1_output_root_mean_squared_error: 0.8940 - val_y2_output_root_mean_squared_error: 1.7633
Epoch 86/500
62/62 [==============================] - 0s 3ms/step - loss: 3.7261 - y1_output_loss: 0.9080 - y2_output_loss: 2.8181 - y1_output_root_mean_squared_error: 0.9529 - y2_output_root_mean_squared_error: 1.6787 - val_loss: 2.7286 - val_y1_output_loss: 0.6998 - val_y2_output_loss: 2.0288 - val_y1_output_root_mean_squared_error: 0.8365 - val_y2_output_root_mean_squared_error: 1.4244
Epoch 87/500
62/62 [==============================] - 0s 3ms/step - loss: 4.5569 - y1_output_loss: 1.3043 - y2_output_loss: 3.2526 - y1_output_root_mean_squared_error: 1.1421 - y2_output_root_mean_squared_error: 1.80

...

Epoch 400/500
62/62 [==============================] - 0s 3ms/step - loss: 0.4736 - y1_output_loss: 0.1209 - y2_output_loss: 0.3526 - y1_output_root_mean_squared_error: 0.3478 - y2_output_root_mean_squared_error: 0.5938 - val_loss: 1.0485 - val_y1_output_loss: 0.2090 - val_y2_output_loss: 0.8395 - val_y1_output_root_mean_squared_error: 0.4571 - val_y2_output_root_mean_squared_error: 0.9163
Epoch 401/500
62/62 [==============================] - 0s 3ms/step - loss: 0.8173 - y1_output_loss: 0.1963 - y2_output_loss: 0.6210 - y1_output_root_mean_squared_error: 0.4431 - y2_output_root_mean_squared_error: 0.7880 - val_loss: 5.5478 - val_y1_output_loss: 0.7476 - val_y2_output_loss: 4.8002 - val_y1_output_root_mean_squared_error: 0.8647 - val_y2_output_root_mean_squared_error: 2.1909
Epoch 402/500
62/62 [==============================] - 0s 3ms/step - loss: 0.7994 - y1_output_loss: 0.2098 - y2_output_loss: 0.5896 - y1_output_root_mean_squared_error: 0.4580 - y2_output_root_mean_squared_error: 0

Epoch 421/500
62/62 [==============================] - 0s 3ms/step - loss: 0.6200 - y1_output_loss: 0.1660 - y2_output_loss: 0.4540 - y1_output_root_mean_squared_error: 0.4075 - y2_output_root_mean_squared_error: 0.6738 - val_loss: 1.2243 - val_y1_output_loss: 0.2289 - val_y2_output_loss: 0.9954 - val_y1_output_root_mean_squared_error: 0.4784 - val_y2_output_root_mean_squared_error: 0.9977
Epoch 422/500
62/62 [==============================] - 0s 4ms/step - loss: 0.4927 - y1_output_loss: 0.1216 - y2_output_loss: 0.3711 - y1_output_root_mean_squared_error: 0.3487 - y2_output_root_mean_squared_error: 0.6092 - val_loss: 1.0385 - val_y1_output_loss: 0.2194 - val_y2_output_loss: 0.8191 - val_y1_output_root_mean_squared_error: 0.4684 - val_y2_output_root_mean_squared_error: 0.9051
Epoch 423/500
62/62 [==============================] - 0s 4ms/step - loss: 0.4310 - y1_output_loss: 0.1274 - y2_output_loss: 0.3036 - y1_output_root_mean_squared_error: 0.3569 - y2_output_root_mean_squared_error: 0

Epoch 442/500
62/62 [==============================] - 0s 4ms/step - loss: 0.3238 - y1_output_loss: 0.1089 - y2_output_loss: 0.2149 - y1_output_root_mean_squared_error: 0.3300 - y2_output_root_mean_squared_error: 0.4636 - val_loss: 1.0686 - val_y1_output_loss: 0.2429 - val_y2_output_loss: 0.8257 - val_y1_output_root_mean_squared_error: 0.4928 - val_y2_output_root_mean_squared_error: 0.9087
Epoch 443/500
62/62 [==============================] - 0s 3ms/step - loss: 0.3726 - y1_output_loss: 0.1082 - y2_output_loss: 0.2644 - y1_output_root_mean_squared_error: 0.3290 - y2_output_root_mean_squared_error: 0.5142 - val_loss: 2.2587 - val_y1_output_loss: 0.2482 - val_y2_output_loss: 2.0105 - val_y1_output_root_mean_squared_error: 0.4982 - val_y2_output_root_mean_squared_error: 1.4179
Epoch 444/500
62/62 [==============================] - 0s 4ms/step - loss: 0.7200 - y1_output_loss: 0.1519 - y2_output_loss: 0.5681 - y1_output_root_mean_squared_error: 0.3898 - y2_output_root_mean_squared_error: 0

Epoch 463/500
62/62 [==============================] - 0s 3ms/step - loss: 0.4294 - y1_output_loss: 0.1224 - y2_output_loss: 0.3071 - y1_output_root_mean_squared_error: 0.3498 - y2_output_root_mean_squared_error: 0.5542 - val_loss: 0.9666 - val_y1_output_loss: 0.2169 - val_y2_output_loss: 0.7497 - val_y1_output_root_mean_squared_error: 0.4657 - val_y2_output_root_mean_squared_error: 0.8659
Epoch 464/500
62/62 [==============================] - 0s 3ms/step - loss: 0.5521 - y1_output_loss: 0.1554 - y2_output_loss: 0.3967 - y1_output_root_mean_squared_error: 0.3942 - y2_output_root_mean_squared_error: 0.6298 - val_loss: 1.5143 - val_y1_output_loss: 0.3194 - val_y2_output_loss: 1.1949 - val_y1_output_root_mean_squared_error: 0.5652 - val_y2_output_root_mean_squared_error: 1.0931
Epoch 465/500
62/62 [==============================] - 0s 3ms/step - loss: 0.5548 - y1_output_loss: 0.1394 - y2_output_loss: 0.4154 - y1_output_root_mean_squared_error: 0.3734 - y2_output_root_mean_squared_error: 0

Epoch 484/500
62/62 [==============================] - 0s 3ms/step - loss: 0.4319 - y1_output_loss: 0.1225 - y2_output_loss: 0.3095 - y1_output_root_mean_squared_error: 0.3499 - y2_output_root_mean_squared_error: 0.5563 - val_loss: 0.9419 - val_y1_output_loss: 0.2498 - val_y2_output_loss: 0.6921 - val_y1_output_root_mean_squared_error: 0.4998 - val_y2_output_root_mean_squared_error: 0.8319
Epoch 485/500
62/62 [==============================] - 0s 4ms/step - loss: 0.3720 - y1_output_loss: 0.1126 - y2_output_loss: 0.2593 - y1_output_root_mean_squared_error: 0.3356 - y2_output_root_mean_squared_error: 0.5093 - val_loss: 0.9927 - val_y1_output_loss: 0.2074 - val_y2_output_loss: 0.7853 - val_y1_output_root_mean_squared_error: 0.4554 - val_y2_output_root_mean_squared_error: 0.8862
Epoch 486/500
62/62 [==============================] - 0s 3ms/step - loss: 0.4556 - y1_output_loss: 0.1334 - y2_output_loss: 0.3221 - y1_output_root_mean_squared_error: 0.3653 - y2_output_root_mean_squared_error: 0

In [19]:
loss, Y1_loss, Y2_loss, Y1_rmse, Y2_rmse = model.evaluate(x=norm_test_x, y=test_y)
print("Loss = {}, Y1_loss = {}, Y1_mse = {}, Y2_loss = {}, Y2_mse = {}".format(loss, Y1_loss, Y1_rmse, Y2_loss, Y2_rmse))

5/5 [==============================] - 0s 1ms/step - loss: 1.0723 - y1_output_loss: 0.2776 - y2_output_loss: 0.7947 - y1_output_root_mean_squared_error: 0.5269 - y2_output_root_mean_squared_error: 0.8915
Loss = 1.0723320245742798, Y1_loss = 0.2776487171649933, Y1_mse = 0.5269238352775574, Y2_loss = 0.7946832776069641, Y2_mse = 0.8914501070976257
